# Preprocessing

In [ ]:
import pandas as pd
import numpy as np
import os
import yaml
from tqdm.notebook import tqdm
from pathlib import Path
from dateutil.relativedelta import relativedelta

In [ ]:
dataset_name = "name_of_your_dataset"
path = "/path/to/mapping/files"
data_path = "/path/to/decoded/output"
dataset_path = f"{data_path}/2_datasets_pre/{dataset_name}"

In [ ]:
Path(dataset_path).mkdir(parents=True, exist_ok=True)


In [ ]:
data = pd.read_feather(f"{data_path}/1_decoded/ukb_data.feather")
data_field = pd.read_feather(f"{data_path}/1_decoded/ukb_data_field.feather")
data_columns = data.columns.to_list()

## Mappings + Vocabulary

In [ ]:
# Drop obvious missing data
print(len(data))
data = data.dropna(subset=["sex_f31_0_0"], axis=0)
print(len(data))

# Starting information

In [ ]:
time0_col="date_of_attending_assessment_centre_f53_0_0"

# Baseline covariates

In [ ]:
def get_fields(fields, data, data_field):
    f = data_field[data_field["field.showcase"].isin(fields) & data_field["field.tab"].str.contains("f\\.\\d+\\.0\\.\\d")].copy()
    f["field"] = pd.Categorical(f["field.showcase"], categories=fields, ordered=True)
    f = f.sort_values("field").reset_index().drop("field", axis=1)
    return f

def get_fields_all(fields, data, data_field):
    f = data_field[data_field["field.showcase"].isin(fields)].copy()
    f["field"] = pd.Categorical(f["field.showcase"], categories=fields, ordered=True)
    f = f.sort_values("field").reset_index().drop("field", axis=1)
    return f

def get_data_fields(fields, data, data_field):
    f = get_fields(fields, data, data_field)
    return data[["eid"]+f["col.name"].to_list()].copy()

def get_data_fields_all(fields, data, data_field):
    f = get_fields_all(fields, data, data_field)
    return data[["eid"]+f["col.name"].to_list()].copy()

### Basics

In [ ]:
coding10 = pd.read_csv(f"{data_path}/mapping/codings/coding10.tsv", sep="\t").assign(coding = lambda x: x.coding.astype("int")).rename(columns={"coding":"uk_biobank_assessment_centre_f54_0_0"})
coding10["uk_biobank_assessment_centre_f54_0_0"] = coding10["uk_biobank_assessment_centre_f54_0_0"].astype("int")

In [ ]:
fields_basics = [
    "21022", # age at recruitment
    "31", # sex
    "21000", # ethnicity
    "189", # Townsend index
    "53", # date of baseline assessment
    "54", # assessment center
]

temp = get_data_fields(fields_basics, data, data_field)

temp["sex_f31_0_0"] = temp["sex_f31_0_0"].cat.set_categories(["Female", 'Male'], ordered=False)

ethn_bg_def = {"White": ["White", "British", "Irish", "Any other white background"],
                "Mixed": ["Mixed", "White and Black Caribbean", "White and Black African", "White and Asian", "Any other mixed background"],  
                "Asian": ["Asian or Asian British", "Indian", "Pakistani", "Bangladeshi", "Any other Asian background"], 
                "Black": ["Black or Black British", "Caribbean", "African", "Any other Black background"],
                "Chinese": ["Chinese"],  
                np.nan: ["Other ethnic group", "Do not know", "Prefer not to answer"]}

ethn_bg_dict = {}
for key, values in ethn_bg_def.items(): 
    for value in values:
        ethn_bg_dict[value]=key 
        
temp["ethnic_background_f21000_0_0"].replace(ethn_bg_dict, inplace=True)
temp["ethnic_background_f21000_0_0"] = temp["ethnic_background_f21000_0_0"].astype("category")

basics = temp

calc_birth_date = [date_of_attending_assessment_centre - relativedelta(years=age_at_recruitment)
                                                             for date_of_attending_assessment_centre, age_at_recruitment 
                                                             in zip(basics["date_of_attending_assessment_centre_f53_0_0"], basics["age_at_recruitment_f21022_0_0"])]

basics = basics.assign(birth_date = calc_birth_date)
basics["uk_biobank_assessment_centre_f54_0_0"] = basics.assign(uk_biobank_assessment_centre_f54_0_0 = lambda x: x.uk_biobank_assessment_centre_f54_0_0.astype("int")).merge(coding10, on="uk_biobank_assessment_centre_f54_0_0")["meaning"]

basics.to_feather(os.path.join(path, dataset_path, 'temp_basics.feather'))

### Questionnaire

In [ ]:
fields_questionnaire = [
    "2178", # Overall health
    "20116", # Smoking status
    "1558",
]

temp = get_data_fields(fields_questionnaire, data, data_field)

temp["overall_health_rating_f2178_0_0"] = temp["overall_health_rating_f2178_0_0"]\
    .replace({"Do not know": np.nan, "Prefer not to answer": np.nan})\
    .astype("category").cat.set_categories(['Poor', 'Fair', 'Good', 'Excellent'], ordered=True)


temp["smoking_status_f20116_0_0"] = temp["smoking_status_f20116_0_0"]\
    .replace({"Prefer not to answer": np.nan}, inplace=False)\
    .astype("category").cat.set_categories(['Current', 'Previous', 'Never'], ordered=True)

temp["alcohol_intake_frequency_f1558_0_0"] = temp["alcohol_intake_frequency_f1558_0_0"]\
    .replace({"Prefer not to answer": np.nan}, inplace=False)\
    .astype("category").cat.set_categories([
        'Daily or almost daily', 
        'Three or four times a week', 
        'Once or twice a week',
        'One to three times a month',
        'Special occasions only', 
        'Never'], ordered=True)

questionnaire = temp

questionnaire.to_feather(os.path.join(path, dataset_path, 'temp_questionnaire.feather'))


### Physical measurements

In [ ]:

fields_measurements = [
    "21001", # BMI
    "21002", # weight
    "4080", # Syst. BP
    "4079", # Diast. BP
    "102",
    "21021",
    "4195",
    "48",
    "49",
    "50",
    "23127",
    "23099",
    "23105",
    "20151",
    "20150",
    "20258",
    "3064",
    
]
temp = get_data_fields(fields_measurements, data, data_field)

sbp_cols = ["systolic_blood_pressure_automated_reading_f4080_0_0", "systolic_blood_pressure_automated_reading_f4080_0_1"]
dbp_cols = ["diastolic_blood_pressure_automated_reading_f4079_0_0", "diastolic_blood_pressure_automated_reading_f4079_0_1"]
pr_cols = ["pulse_rate_automated_reading_f102_0_0", "pulse_rate_automated_reading_f102_0_1"]

temp = temp.assign(systolic_blood_pressure_automated_reading_f4080 = temp[sbp_cols].mean(axis=1),
                   diastolic_blood_pressure_automated_reading_f4079 = temp[dbp_cols].mean(axis=1),
                   pulse_rate_automated_reading_f102 = temp[pr_cols].mean(axis=1))\
    .drop(sbp_cols + dbp_cols + pr_cols, axis=1)

measurements = temp

measurements.to_feather(os.path.join(path, dataset_path, 'temp_measurements.feather'))

### Lab measurements

In [ ]:
fields_blood_count = [
    "30160", #	Basophill count
    "30220", #	Basophill percentage
    "30150", #	Eosinophill count
    "30210", #	Eosinophill percentage
    "30030", #	Haematocrit percentage
    "30020", #	Haemoglobin concentration
    "30300", #	High light scatter reticulocyte count
    "30290", #	High light scatter reticulocyte percentage
    "30280", #	Immature reticulocyte fraction
    "30120", #	Lymphocyte count
    "30180", #	Lymphocyte percentage
    "30050", #	Mean corpuscular haemoglobin
    "30060", #	Mean corpuscular haemoglobin concentration
    "30040", #	Mean corpuscular volume
    "30100", #	Mean platelet (thrombocyte) volume
    "30260", #	Mean reticulocyte volume
    "30270", #	Mean sphered cell volume
    "30130", #	Monocyte count
    "30190", #	Monocyte percentage
    "30140", #	Neutrophill count
    "30200", #	Neutrophill percentage
    "30170", #	Nucleated red blood cell count
    "30230", #	Nucleated red blood cell percentage
    "30080", #	Platelet count
    "30090", #	Platelet crit
    "30110", #	Platelet distribution width
    "30010", #	Red blood cell (erythrocyte) count
    "30070", #	Red blood cell (erythrocyte) distribution width
    "30250", #	Reticulocyte count
    "30240", #	Reticulocyte percentage
    "30000", #	White blood cell (leukocyte) count
]

fields_blood_biochemistry = [
    "30620",#	Alanine aminotransferase
    "30600",#	Albumin
    "30610",#	Alkaline phosphatase
    "30630",#	Apolipoprotein A
    "30640",#	Apolipoprotein B
    "30650",#	Aspartate aminotransferase
    "30710",#	C-reactive protein
    "30680",#	Calcium
    "30690",#	Cholesterol
    "30700",#	Creatinine
    "30720",#	Cystatin C
    "30660",#	Direct bilirubin
    "30730",#	Gamma glutamyltransferase
    "30740",#	Glucose
    "30750",#	Glycated haemoglobin (HbA1c)
    "30760",#	HDL cholesterol
    "30770",#	IGF-1
    "30780",#	LDL direct
    "30790",#	Lipoprotein A
    "30800",#	Oestradiol
    "30810",#	Phosphate
    "30820",#	Rheumatoid factor
    "30830",#	SHBG
    "30850",#	Testosterone
    "30840",#	Total bilirubin
    "30860",#	Total protein
    "30870",#	Triglycerides
    "30880",#	Urate
    "30670",#	Urea
    "30890",#	Vitamin D
]

fields_blood_infectious = [
    "23000", #	1gG antigen for Herpes Simplex virus-1
    "23001", #	2mgG unique antigen for Herpes Simplex virus-2
    "23049", #	Antigen assay QC indicator
    "23048", #	Antigen assay date
    "23026", #	BK VP1 antigen for Human Polyomavirus BKV
    "23039", #	CagA antigen for Helicobacter pylori
    "23043", #	Catalase antigen for Helicobacter pylori
    "23018", #	Core antigen for Hepatitis C Virus
    "23030", #	E6 antigen for Human Papillomavirus type-16
    "23031", #	E7 antigen for Human Papillomavirus type-16
    "23006", #	EA-D antigen for Epstein-Barr Virus
    "23004", #	EBNA-1 antigen for Epstein-Barr Virus
    "23042", #	GroEL antigen for Helicobacter pylori
    "23016", #	HBc antigen for Hepatitis B Virus
    "23017", #	HBe antigen for Hepatitis B Virus
    "23025", #	HIV-1 env antigen for Human Immunodeficiency Virus
    "23024", #	HIV-1 gag antigen for Human Immunodeficiency Virus
    "23023", #	HTLV-1 env antigen for Human T-Lymphotropic Virus 1
    "23022", #	HTLV-1 gag antigen for Human T-Lymphotropic Virus 1
    "23010", #	IE1A antigen for Human Herpesvirus-6
    "23011", #	IE1B antigen for Human Herpesvirus-6
    "23027", #	JC VP1 antigen for Human Polyomavirus JCV
    "23015", #	K8.1 antigen for Kaposi's Sarcoma-Associated Herpesvirus
    "23029", #	L1 antigen for Human Papillomavirus type-16
    "23032", #	L1 antigen for Human Papillomavirus type-18
    "23014", #	LANA antigen for Kaposi's Sarcoma-Associated Herpesvirus
    "23028", #	MC VP1 antigen for Merkel Cell Polyomavirus
    "23019", #	NS3 antigen for Hepatitis C Virus
    "23041", #	OMP antigen for Helicobacter pylori
    "23037", #	PorB antigen for Chlamydia trachomatis
    "23013", #	U14 antigen for Human Herpesvirus-7
    "23044", #	UreA antigen for Helicobacter pylori
    "23003", #	VCA p18 antigen for Epstein-Barr Virus
    "23040", #	VacA antigen for Helicobacter pylori
    "23005", #	ZEBRA antigen for Epstein-Barr Virus
    "23002", #	gE / gI antigen for Varicella Zoster Virus
    "23034", #	momp A antigen for Chlamydia trachomatis
    "23033", #	momp D antigen for Chlamydia trachomatis
    "23012", #	p101 k antigen for Human Herpesvirus-6
    "23020", #	p22 antigen for Toxoplasma gondii
    "23038", #	pGP3 antigen for Chlamydia trachomatis
    "23009", #	pp 28 antigen for Human Cytomegalovirus
    "23008", #	pp 52 antigen for Human Cytomegalovirus
    "23007", #	pp150 Nter antigen for Human Cytomegalovirus
    "23021", #	sag1 antigen for Toxoplasma gondii
    "23035", #	tarp-D F1 antigen for Chlamydia trachomatis
    "23036", #	tarp-D F2 antigen for Chlamydia trachomatis
]

labs = temp = get_data_fields(fields_blood_count+fields_blood_biochemistry+fields_blood_infectious, data, data_field)

labs.to_feather(os.path.join(path, dataset_path, 'temp_labs.feather'))

In [ ]:
fields_metabolomics = [
"23474", #3-Hydroxybutyrate
"23475", #Acetate
"23476", #Acetoacetate
"23477", #Acetone
"23460", #Alanine
"23479", #Albumin
"23440", #Apolipoprotein A1
"23439", #Apolipoprotein B
"23433", #Average Diameter for HDL Particles
"23432", #Average Diameter for LDL Particles
"23431", #Average Diameter for VLDL Particles
"23484", #Cholesterol in Chylomicrons and Extremely Large VLDL
"23526", #Cholesterol in IDL
"23561", #Cholesterol in Large HDL
"23533", #Cholesterol in Large LDL
"23498", #Cholesterol in Large VLDL
"23568", #Cholesterol in Medium HDL
"23540", #Cholesterol in Medium LDL
"23505", #Cholesterol in Medium VLDL
"23575", #Cholesterol in Small HDL
"23547", #Cholesterol in Small LDL
"23512", #Cholesterol in Small VLDL
"23554", #Cholesterol in Very Large HDL
"23491", #Cholesterol in Very Large VLDL
"23519", #Cholesterol in Very Small VLDL
"23485", #Cholesteryl Esters in Chylomicrons and Extremely Large VLDL
"23418", #Cholesteryl Esters in HDL
"23527", #Cholesteryl Esters in IDL
"23417", #Cholesteryl Esters in LDL
"23562", #Cholesteryl Esters in Large HDL
"23534", #Cholesteryl Esters in Large LDL
"23499", #Cholesteryl Esters in Large VLDL
"23569", #Cholesteryl Esters in Medium HDL
"23541", #Cholesteryl Esters in Medium LDL
"23506", #Cholesteryl Esters in Medium VLDL
"23576", #Cholesteryl Esters in Small HDL
"23548", #Cholesteryl Esters in Small LDL
"23513", #Cholesteryl Esters in Small VLDL
"23416", #Cholesteryl Esters in VLDL
"23555", #Cholesteryl Esters in Very Large HDL
"23492", #Cholesteryl Esters in Very Large VLDL
"23520", #Cholesteryl Esters in Very Small VLDL
"23473", #Citrate
"23404", #Clinical LDL Cholesterol
"23481", #Concentration of Chylomicrons and Extremely Large VLDL Particles
"23430", #Concentration of HDL Particles
"23523", #Concentration of IDL Particles
"23429", #Concentration of LDL Particles
"23558", #Concentration of Large HDL Particles
"23530", #Concentration of Large LDL Particles
"23495", #Concentration of Large VLDL Particles
"23565", #Concentration of Medium HDL Particles
"23537", #Concentration of Medium LDL Particles
"23502", #Concentration of Medium VLDL Particles
"23572", #Concentration of Small HDL Particles
"23544", #Concentration of Small LDL Particles
"23509", #Concentration of Small VLDL Particles
"23428", #Concentration of VLDL Particles
"23551", #Concentration of Very Large HDL Particles
"23488", #Concentration of Very Large VLDL Particles
"23516", #Concentration of Very Small VLDL Particles
"23478", #Creatinine
"23443", #Degree of Unsaturation
"23450", #Docosahexaenoic Acid
"23486", #Free Cholesterol in Chylomicrons and Extremely Large VLDL
"23422", #Free Cholesterol in HDL
"23528", #Free Cholesterol in IDL
"23421", #Free Cholesterol in LDL
"23563", #Free Cholesterol in Large HDL
"23535", #Free Cholesterol in Large LDL
"23500", #Free Cholesterol in Large VLDL
"23570", #Free Cholesterol in Medium HDL
"23542", #Free Cholesterol in Medium LDL
"23507", #Free Cholesterol in Medium VLDL
"23577", #Free Cholesterol in Small HDL
"23549", #Free Cholesterol in Small LDL
"23514", #Free Cholesterol in Small VLDL
"23420", #Free Cholesterol in VLDL
"23556", #Free Cholesterol in Very Large HDL
"23493", #Free Cholesterol in Very Large VLDL
"23521", #Free Cholesterol in Very Small VLDL
"23470", #Glucose
"23461", #Glutamine
"23462", #Glycine
"23480", #Glycoprotein Acetyls
"23406", #HDL Cholesterol
"23463", #Histidine
"23465", #Isoleucine
"23405", #LDL Cholesterol
"23471", #Lactate
"23466", #Leucine
"23449", #Linoleic Acid
"23447", #Monounsaturated Fatty Acids
"23444", #Omega-3 Fatty Acids
"23445", #Omega-6 Fatty Acids
"23468", #Phenylalanine
"23437", #Phosphatidylcholines
"23434", #Phosphoglycerides
"23483", #Phospholipids in Chylomicrons and Extremely Large VLDL
"23414", #Phospholipids in HDL
"23525", #Phospholipids in IDL
"23413", #Phospholipids in LDL
"23560", #Phospholipids in Large HDL
"23532", #Phospholipids in Large LDL
"23497", #Phospholipids in Large VLDL
"23567", #Phospholipids in Medium HDL
"23539", #Phospholipids in Medium LDL
"23504", #Phospholipids in Medium VLDL
"23574", #Phospholipids in Small HDL
"23546", #Phospholipids in Small LDL
"23511", #Phospholipids in Small VLDL
"23412", #Phospholipids in VLDL
"23553", #Phospholipids in Very Large HDL
"23490", #Phospholipids in Very Large VLDL
"23518", #Phospholipids in Very Small VLDL
"23446", #Polyunsaturated Fatty Acids
"23472", #Pyruvate
"23402", #Remnant Cholesterol (Non-HDL, Non-LDL -Cholesterol)
"23448", #Saturated Fatty Acids
"23438", #Sphingomyelins
"23400", #Total Cholesterol
"23401", #Total Cholesterol Minus HDL-C
"23436", #Total Cholines
"23464", #Total Concentration of Branched-Chain Amino Acids (Leucine + Isoleucine + Valine)
"23427", #Total Concentration of Lipoprotein Particles
"23415", #Total Esterified Cholesterol
"23442", #Total Fatty Acids
"23419", #Total Free Cholesterol
"23482", #Total Lipids in Chylomicrons and Extremely Large VLDL
"23426", #Total Lipids in HDL
"23524", #Total Lipids in IDL
"23425", #Total Lipids in LDL
"23559", #Total Lipids in Large HDL
"23531", #Total Lipids in Large LDL
"23496", #Total Lipids in Large VLDL
"23423", #Total Lipids in Lipoprotein Particles
"23566", #Total Lipids in Medium HDL
"23538", #Total Lipids in Medium LDL
"23503", #Total Lipids in Medium VLDL
"23573", #Total Lipids in Small HDL
"23545", #Total Lipids in Small LDL
"23510", #Total Lipids in Small VLDL
"23424", #Total Lipids in VLDL
"23552", #Total Lipids in Very Large HDL
"23489", #Total Lipids in Very Large VLDL
"23517", #Total Lipids in Very Small VLDL
"23411", #Total Phospholipids in Lipoprotein Particles
"23407", #Total Triglycerides
"23487", #Triglycerides in Chylomicrons and Extremely Large VLDL
"23410", #Triglycerides in HDL
"23529", #Triglycerides in IDL
"23409", #Triglycerides in LDL
"23564", #Triglycerides in Large HDL
"23536", #Triglycerides in Large LDL
"23501", #Triglycerides in Large VLDL
"23571", #Triglycerides in Medium HDL
"23543", #Triglycerides in Medium LDL
"23508", #Triglycerides in Medium VLDL
"23578", #Triglycerides in Small HDL
"23550", #Triglycerides in Small LDL
"23515", #Triglycerides in Small VLDL
"23408", #Triglycerides in VLDL
"23557", #Triglycerides in Very Large HDL
"23494", #Triglycerides in Very Large VLDL
"23522", #Triglycerides in Very Small VLDL
"23469", #Tyrosine
"23403", #VLDL Cholesterol
"23467", #Valine
    "23651", "23650"
]

metabolomics = temp = get_data_fields(fields_metabolomics, data, data_field)
metabolomics.columns = ["eid"]+[f"NMR_{col}" for col in metabolomics.columns[1:]]
metabolomics["NMR_FLAG"] = [True if type(spectrometer)==str else False for spectrometer in metabolomics["NMR_spectrometer_f23650_0_0"].tolist()]


metabolomics.to_feather(os.path.join(path, dataset_path, 'temp_metabolomics.feather'))

### Family History

In [ ]:
fh_list=["Heart disease", "Stroke", "High blood pressure",  "Diabetes", "Lung cancer", "Severe depression", "Parkinson's disease", "Alzheimer's disease/dementia", "Chronic bronchitis/emphysema", "Breast cancer", "Bowel cancer"]
with open(os.path.join(path, dataset_path, 'fh_list.yaml'), 'w') as file: yaml.dump(fh_list, file, default_flow_style=False)

fields_family_history = [
    "20107", # Family history 
    "20110" # Family history
]

raw = get_data_fields(fields_family_history, data, data_field)
temp = pd.melt(raw, id_vars=["eid"], value_vars=raw.drop("eid", axis=1).columns.to_list(), var_name = "field", value_name="family_history").drop("field", axis=1)
temp = temp[temp.family_history.isin(fh_list)].assign(family_history=temp["family_history"].str.lower().replace(" ", "_", regex=True))

temp = temp.drop_duplicates().sort_values("eid").reset_index().drop("index", axis=1).assign(n=True)
temp = pd.pivot_table(temp, index="eid", columns="family_history", values="n", observed=True).add_prefix('fh_')
family_history = temp = data[["eid"]].copy().merge(temp, how="left", on="eid").fillna(False)


family_history.to_feather(os.path.join(path, dataset_path, 'temp_family_history.feather'))

## Medications

In [ ]:
# https://list.essentialmeds.org/?showRemoved=0
# essential medicines WHO?!

In [ ]:
atc_mapping = pd.read_csv(f"{data_path}/mapping/atc/atc_matched_list.csv")
athena_concepts = pd.read_csv(f"{data_path}/athena_vocabulary/CONCEPT.csv", sep="\t").assign(vocabulary_id = lambda x: x.vocabulary_id.astype("string"), concept_class_id = lambda x: x.concept_class_id.astype("string"))
atc_concepts = athena_concepts[athena_concepts.vocabulary_id=="ATC"]
atc2_concepts = atc_concepts[atc_concepts.concept_class_id=="ATC 2nd"].sort_values("concept_code")
medication_list = dict(zip([x.lower().replace(" ", "_") for x in atc2_concepts.concept_name.to_list()], [[x] for x in atc2_concepts.concept_code.to_list()]))
medication_list_extra = {
    "antihypertensives": ["C02"],
    "statins": ["C10A", "C10B"],
    "ass": ["B01"],
    "atypical_antipsychotics" : ["N05"],
    "glucocorticoids" : ["H02"]                        
}
medication_list.update(medication_list_extra)

with open(os.path.join(path, dataset_path, 'medication_list.yaml'), 'w') as file: yaml.dump(medication_list, file, default_flow_style=False)

In [ ]:
def had_medication_before(data, data_field, medications, atc_mapping):
    fields = ["20003"]
    raw = get_data_fields(fields, data, data_field)
    temp = pd.melt(raw, id_vars=["eid"], value_vars=raw.drop("eid", axis=1).columns.to_list(), var_name = "field", value_name="UKBB_code").drop("field", axis=1).drop_duplicates()

    temp.UKBB_code = temp.UKBB_code.astype(str)
    temp = temp[temp.UKBB_code!="None"].copy()
    temp = temp[temp.UKBB_code!="nan"].copy()
    temp.UKBB_code = temp.UKBB_code.astype(int)

    temp_atc = temp.merge(atc_mapping, how="left", on="UKBB_code").sort_values("eid").reset_index(drop=True).dropna(subset=["ATC_code"], axis=0)
    temp_atc.ATC_code = temp_atc.ATC_code.astype("string")
    temp = data[["eid"]].copy()
    for med, med_codes in tqdm(medication_list.items()):
        regex_str = "^"+"|^".join(med_codes)
        df = temp_atc[temp_atc.ATC_code.str.contains(regex_str, case=False)][["eid"]]\
            .drop_duplicates(subset=["eid"])\
            .assign(medication=True)
        temp[med] = temp.merge(df, how="left", on="eid").fillna(False).medication
        
    return temp.sort_values("eid")

In [ ]:
medications = had_medication_before(data, data_field, medication_list, atc_mapping)

medications.to_feather(os.path.join(path, dataset_path, 'temp_medications.feather'))

## Diagnoses and events

In [ ]:
vocab_dir = f"{data_path}/mapping/athena"
vocab = {
    "concept": pd.read_csv(f"{vocab_dir}/CONCEPT.csv", sep='\t'),
    "domain": pd.read_csv(f"{vocab_dir}/DOMAIN.csv", sep='\t'),
    "class": pd.read_csv(f"{vocab_dir}/CONCEPT_CLASS.csv", sep='\t'),
    "relationship": pd.read_csv(f"{vocab_dir}/RELATIONSHIP.csv", sep='\t'),
    "drug_strength": pd.read_csv(f"{vocab_dir}/DRUG_STRENGTH.csv", sep='\t'),
    "vocabulary": pd.read_csv(f"{vocab_dir}/VOCABULARY.csv", sep='\t'),
    "concept_synonym": pd.read_csv(f"{vocab_dir}/CONCEPT_SYNONYM.csv", sep='\t'),
    "concept_ancestor": pd.read_csv(f"{vocab_dir}/CONCEPT_ANCESTOR.csv", sep='\t'),
    "concept_relationship": pd.read_csv(f"{vocab_dir}/CONCEPT_RELATIONSHIP.csv", sep='\t')                       
}

### Definitions

In [ ]:
coding1836 = pd.read_csv(f"{data_path}/mapping/codings/coding1836.tsv", sep="\t").rename(columns={"coding":"code"})
phecodes = pd.read_csv(f"{data_path}/mapping/phecodes/phecode_icd10.csv")
def phenotype_children(phecodes, phenotype_list):
    l={}
    phecodes = phecodes.dropna(subset=["Phenotype"], axis=0)
    for ph, ph_names in phenotype_list.items():
        regex = "|".join(ph_names)
        l[ph] = list(phecodes[phecodes.Phenotype.str.contains(regex, case=False)].ICD10.str.replace("\\.", "").str.slice(0, 3).unique())
    return l

In [ ]:
snomed_core = pd.read_csv(f"{data_path}/mapping/snomed_core_list.txt", sep="|")

In [ ]:
snomed_core = snomed_core.query("SNOMED_CONCEPT_STATUS == 'Current'").copy()
new = snomed_core.SNOMED_FSN.str.split("(", n=1, expand=True)
snomed_core["snomed_name"] = new[0].str.rstrip(' ')
snomed_core["snomed_type"] = new[1].str.rstrip(')')
snomed_core_data = snomed_core

In [ ]:
snomed_names = snomed_core_data.snomed_name.to_list()
snomed_names = [str(item).lower().strip().replace(" ", "_").replace(";", "").replace(",", "") for item in snomed_names]

In [ ]:
phenotype_list_snomed = dict(zip(snomed_names, snomed_core_data.SNOMED_CID.to_list()))
snomed_df = pd.DataFrame.from_dict(phenotype_list_snomed, orient='index').reset_index()
snomed_df.columns = ["diagnosis", "concept_code"]

In [ ]:
concept_ids = vocab["concept"].query("(vocabulary_id == 'SNOMED') | (vocabulary_id == 'ICD10CM')")
concept_ids_icd10 = vocab["concept"].query("vocabulary_id == 'ICD10CM'").concept_id.to_list()
vocab_concept_ids = concept_ids.concept_id.to_list()
concept_ancestor = vocab["concept_ancestor"][["ancestor_concept_id", "descendant_concept_id"]].query("ancestor_concept_id == @vocab_concept_ids")

In [ ]:
concept_rel = vocab["concept_relationship"][["concept_id_1", "concept_id_2", "relationship_id"]].query("(concept_id_1 == @vocab_concept_ids) & (concept_id_2 == @concept_ids_icd10) & (relationship_id == 'Mapped from')")
concept_mapping = concept_rel.rename(columns={"concept_id_2":"concept_id"}).merge(concept_ids, on="concept_id").query("vocabulary_id == 'ICD10CM'")[["concept_id_1", "concept_code"]].rename(columns={"concept_id_1":"concept_id_desc","concept_code":"icd10"})

In [ ]:
df_snomed_concept_id = snomed_df.merge(concept_ids[["concept_code", "concept_id"]], on="concept_code")

In [ ]:
df_icd = df_snomed_concept_id.merge(concept_mapping.rename(columns={"concept_id_desc":"concept_id"}), on="concept_id")
df_mapped = df_icd[["diagnosis", "concept_code", "icd10"]].drop_duplicates()
df_mapped["icd10"] = df_mapped["icd10"].str.replace(".", "")
df_mapped["meaning"] = [e[:3] for e in df_mapped["icd10"].to_list()]
icd10_codes = dict(df_mapped[["diagnosis", "meaning"]].drop_duplicates().groupby("diagnosis")["meaning"].apply(list).to_dict())#set_index("diagnosis", drop=True).to_dict()["meaning"]#.sort_values("diagnosis")

In [ ]:
l10_snomed = {}
for ph in icd10_codes: l10_snomed.update({ph:icd10_codes[ph]})

In [ ]:
l10_basic = {
    "myocardial_infarction": ['I21', 'I22', 'I23', 'I24', 'I25'],
    "stroke": ['G45', "I63", "I64"],
    "diabetes1" : ['E10'],
    "diabetes2" : ['E11', 'E12', 'E13', 'E14'],
    "chronic_kidney_disease": ["I12", "N18", "N19"],
    'atrial_fibrillation': ['I47', 'I48'],
    'migraine': ['G43', 'G44'],
    'rheumatoid_arthritis': ['J99', 'M05', 'M06', 'M08', 'M12', 'M13'],
    "systemic_lupus_erythematosus": ['M32'],
    'severe_mental_illness': ['F20', 'F25', 'F30', 'F31', 'F32', 'F33', 'F44'],
    "erectile_dysfunction" : ['F52', 'N48'],  
    "liver_disease":["K70", "K71", "K72", "K73", "K74", "K75", "K76", "K77"],
    "dementia":['F00', 'F01', 'F02', 'F03'],
    "copd": ['J44']}

In [ ]:
l10_metabolomics = {
    "M_all_cause_dementia": ["F00", "F01", "F02", "F03", "G30", "G31"],
    "M_MACE": ["G45", "I21", "I22", "I23", "I24", "I25", "I63", "I64"],
    "M_type_2_diabetes": ["E10", "E11", "E12", "E13", "E14"],
    "M_liver_disease": ["B15", "B16", "B17", "B18", "B19", "C22", "E83", "E88", "I85", 
                          "K70", "K72", "K73", "K74", "K75", "K76", "R18", "Z94"],
    "M_renal_disease":  [f"N{i:02}" for i in range(20)]+[f"N{i:02}" for i in range(25, 30)],
    "M_atrial_fibrillation": ["I48"],
    "M_heart_failure":["I50"],
    "M_coronary_heart_disease": [f"I{i:02}" for i in range(20, 26)],
    "M_venous_thrombosis": ["I80", "I81", "I82"],
    "M_cerebral_stroke":["I63", "I65", "I66"],
    "M_haemorrhagic_stroke": ["I60, I61, I62"],
    "M_abdominal_aortic_aneurysm" : ["I71"],
    "M_peripheral_arterial_disease": ['I70', 'I71', 'I72', 'I73', 'I74', 'I75', 'I76', 'I77', 'I78', 'I79'],
    "M_asthma":["J45", "J46"],
    "M_chronic_obstructuve_pulmonary_disease":["J40", "J41", "J42", "J43", "J44", "J47"],
    "M_lung_cancer":["C33", "C34"],
    "M_non_melanoma_skin_cancer":["C44"],
    "M_stomach_cancer":["C16"],
    "M_oesophagus_cancer":["C15"],
    "M_colon_cancer":["C18"],
    "M_rectal_cancer":["C19", "C20"],
    "M_prostate_cancer":["C61"],
    "M_ovarian_cancer":["C56", "C57"],
    "M_breast_cancer":["C50"],
    "M_uterus_cancer":["C54"],
    "M_parkinsons_disease":["G20", "G21", "G22"],
    "M_fractures":["S02", "S12", "S22", "S32", "S42", "S52", "S62", "S72", "S82", "S92", "T02", "T08", "T10"],
    "M_cataracts":["H25", "H26"],
    "M_glaucoma":["H40"]  
}

In [ ]:
l10_all = l10_basic
for key, value in l10_metabolomics.items(): 
    if key not in l10_basic: l10_all[key] = value
for key, value in l10_snomed.items(): 
    if key not in l10_basic: l10_all[key] = value

In [ ]:
l10 = {k: v for k, v in l10_all.items() if len(v)!=0}

with open(os.path.join(path, dataset_path, 'phenotype_list.yaml'), 'w') as file: yaml.dump(l10, file, default_flow_style=False)

### 1. Self Reported

In [ ]:
coding609 = pd.read_csv(f"{data_path}/mapping/codings/coding609.tsv", sep="\t").rename(columns={"coding":"code"})

In [ ]:
from datetime import datetime, timedelta

def datetime_from_dec_year(dec_year):
    start = dec_year
    year = int(start)
    rem = start - year

    base = datetime(year, 1, 1)
    result = base + timedelta(seconds=(base.replace(year=base.year + 1) - base).total_seconds() * rem)
    #result.strftime("%Y-%m-%d")
    return result.date()

def extract_map_self_reported(data, data_field, code_map):
    pbar = tqdm(total=16)
    ### codes
    fields = ["20002"]; pbar.update(1)
    raw = get_data_fields_all(fields, data, data_field); pbar.update(1)
    col = "noncancer_illness_code_selfreported_f20002"; pbar.update(1)
    temp = pd.wide_to_long(raw, stubnames=[col], i="eid", j="instance_index", sep="_", suffix="\w+").reset_index(); pbar.update(1)
    codes = temp.rename(columns={col:"code"})\
        .assign(code=lambda x: x.code.astype(str))\
        .replace("None", np.nan) \
        .replace("nan", np.nan) \
        .dropna(subset=["code"], axis=0)\
        .assign(code=lambda x: x.code.astype(int)) \
        .merge(code_map, how="left",on="code") \
        .dropna(subset=["meaning"], axis=0)\
        .sort_values(["eid", "instance_index"]) \
        .reset_index(drop=True); pbar.update(1)
    
    ### dates
    fields = ["20008"]; pbar.update(1)
    raw = get_data_fields_all(fields, data, data_field); pbar.update(1)
    col="interpolated_year_when_noncancer_illness_first_diagnosed_f20008"; pbar.update(1)
    temp = pd.wide_to_long(raw, stubnames=[col], i="eid", j="instance_index", sep="_", suffix="\w+").reset_index(); pbar.update(1)
    dates = temp.rename(columns={col:"date"})\
        .dropna(subset=["date"], axis=0)\
        .sort_values(["eid", "instance_index"]) \
        .reset_index(drop=True); pbar.update(1)

    dates = dates[dates.date!=-1]; pbar.update(1)
    dates = dates[dates.date!=-3]; pbar.update(1)
    dates.date = dates.date.apply(datetime_from_dec_year); pbar.update(1)
    
    test = codes.merge(dates, how="left", on=["eid", "instance_index"]).assign(origin="self_reported").copy(); pbar.update(1)
    
    test["instance_index"] = test["instance_index"].astype("string"); pbar.update(1)
    test[['instance','n']] = test.instance_index.str.split("_",expand=True); pbar.update(1)
    pbar.close()
    
    return test[["eid", "origin", 'instance','n', "code", "meaning", "date"]]

In [ ]:
codes_self_reported = extract_map_self_reported(data, data_field, coding609)

### Load records

In [ ]:
codes_gp_records = pd.read_feather(f"{data_path}/1_decoded/codes_gp_diagnoses_210119.feather").drop("level", axis=1)
codes_hospital_records = pd.read_feather(f"{data_path}/1_decoded/codes_hes_diagnoses_210120.feather")

### Combine diagnoses and events

In [ ]:
diagnoses_codes = codes_self_reported.append(codes_hospital_records).append(codes_gp_records).sort_values(["eid", "date"]).dropna(subset=["date"], axis=0).reset_index(drop=True)
diagnoses_codes.head()

In [ ]:
diagnoses_codes.reset_index(drop=True).assign(eid = lambda x: x.eid.astype(int),
                                              origin = lambda x: x.origin.astype(str),
                                              instance = lambda x: x.instance.astype(int),
                                              n = lambda x: x.n.astype(int),
                                              code = lambda x: x.code.astype(str), 
                                              meaning = lambda x: x.meaning.astype(str))\
    .to_feather(os.path.join(path, dataset_path, 'temp_diagnoses_codes.feather'))

In [ ]:
diagnoses_codes = pd.read_feather(os.path.join(path, dataset_path, 'temp_diagnoses_codes.feather'))


In [ ]:
from joblib import Parallel, delayed

def had_diagnosis_before_per_ph(df_before, ph, ph_codes, temp):
    df_ph = df_before[df_before.meaning.isin(ph_codes)][["eid"]]\
            .drop_duplicates(subset=["eid"])\
            .assign(phenotype=True) 
    return temp.merge(df_ph, how="left", on="eid").fillna(False).phenotype

def had_diagnosis_before(data, diagnoses_codes, phenotypes, time0=time0_col):
    diagnoses_codes_time = diagnoses_codes.merge(data[["eid", time0]], how="left", on="eid")
    
    temp = data[["eid"]].copy()
    df_before = diagnoses_codes_time[diagnoses_codes_time.date < diagnoses_codes_time[time0]]
                                                                                         
    df_phs = Parallel(n_jobs=20, require="sharedmem")(delayed(had_diagnosis_before_per_ph)(df_before, ph, phenotypes[ph], temp) for ph in tqdm(list(phenotypes)))
    for ph, df_ph_series in zip(tqdm(list(phenotypes)), df_phs): temp[ph] = df_ph_series
    
    return temp.sort_values("eid")

In [ ]:
diagnoses = had_diagnosis_before(basics, diagnoses_codes, l10, time0=time0_col)

diagnoses.to_feather(os.path.join(path, dataset_path, 'temp_diagnoses.feather'))


In [ ]:
diagnoses = pd.read_feather(os.path.join(path, dataset_path, 'temp_diagnoses.feather'))

## Merge Everything

In [ ]:
data_dfs_dict = {"basics": pd.read_feather(os.path.join(path, dataset_path, 'temp_basics.feather')), 
                 "questionnaire": pd.read_feather(os.path.join(path, dataset_path, 'temp_questionnaire.feather')), 
                 "measurements": pd.read_feather(os.path.join(path, dataset_path, 'temp_measurements.feather')), 
                 "labs": pd.read_feather(os.path.join(path, dataset_path, 'temp_labs.feather')), 
                "metabolomics": pd.read_feather(os.path.join(path, dataset_path, 'temp_metabolomics.feather')), 
                 "family_history": pd.read_feather(os.path.join(path, dataset_path, 'temp_family_history.feather')), 
                 "diagnoses": pd.read_feather(os.path.join(path, dataset_path, 'temp_diagnoses.feather')),
                 "medications": pd.read_feather(os.path.join(path, dataset_path, 'temp_medications.feather'))}

In [ ]:
def get_cols_clean(df):
    df.columns = df.columns.str.replace(r'_0_0$', '').str.replace("_automated_reading", '')
    fields = df.columns.str.extract(r'_f([0-9]+)$').squeeze().to_list()
    df.columns = df.columns.str.replace(r'_f[0-9]+$', '')
    return df.columns, fields

def clean_df(df):
    df.columns, fields = get_cols_clean(df)
    return df, fields

In [ ]:
import pandas as pd
from functools import reduce

data_baseline = reduce(lambda x, y: pd.merge(x, y, on = 'eid'), list(data_dfs_dict.values()))

In [ ]:
data_baseline, fields = clean_df(data_baseline)

In [ ]:
for col in [col for col in list(data_baseline.columns) if ("_event" in col) & ("_time" not in col)]:
    data_baseline[col] = data_baseline[col].astype(int)

In [ ]:
covariates = [col for col in list(data_baseline.columns) if not "_event" in col]
targets = [col for col in list(data_baseline.columns) if "_event" in col]

# Exporting

In [ ]:
data_cols = {}
for topic, df in data_dfs_dict.items(): 
    data_cols["eid"] = ["admin"]
    data_cols[topic]=list(get_cols_clean(df)[0])[1:]

In [ ]:
data_cols_single = {}
for topic, columns in data_cols.items():
    for col in columns:
        data_cols_single[col] = topic

In [ ]:
dtypes = {"int32":"int", "int64":"int", "float64":"float", "category":"category", "object":"category", "bool":"bool"}
desc_dict = {"id": [*range(1, len(data_baseline.columns.to_list())+1)] , 
             "covariate": data_baseline.columns.to_list(), 
             "dtype":[dtypes[str(col)] for col in data_baseline.dtypes.to_list()], 
             "isTarget":[True if col in targets else False for col in data_baseline.columns.to_list()],
            "based_on":[topic for col, topic in data_cols_single.items()],
             "field": fields,
            "aggr_fn": [np.nan for col in data_baseline.columns.to_list()]}
data_baseline_description = pd.DataFrame.from_dict(desc_dict)

# Exclusion Criteria

In [ ]:
feature_dict = {}
for group in data_baseline_description.based_on.unique(): feature_dict[group] = data_baseline_description.query("based_on==@group").covariate.to_list()
with open(os.path.join(path, dataset_path, 'feature_list.yaml'), 'w') as file: yaml.dump(feature_dict, file, default_flow_style=False, allow_unicode=True)

In [ ]:
data_baseline.to_feather(os.path.join(path, dataset_path, 'baseline_covariates.feather'))
data_baseline_description.to_feather(os.path.join(path, dataset_path, 'baseline_covariates_description.feather'))

# !!! REMEMBER IMPUTATION !!!